In [38]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, SimpleRNN, Dropout, LSTM, Embedding, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.constraints import maxnorm

# Fake detection using Passive Agressive Classifier

### Data preparation

In [2]:
df = pd.read_csv("news.csv", nrows = 3000)
df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [3]:
conversion_dict = {'REAL': 0, 'FAKE': 1}
df['label'] = df['label'].replace(conversion_dict)
df.label.value_counts()

1    1511
0    1489
Name: label, dtype: int64

In [4]:
df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",1
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",1
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,0
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,0
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,0


### Words vectorize and model build

In [5]:
x_train_first,x_test_first,y_train_first,y_test_first=train_test_split(df['text'], df['label'], test_size=0.25, random_state=7, shuffle=True)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75 )

In [6]:
first_vec_train=tfidf_vectorizer.fit_transform(x_train_first.values.astype('U'))
first_vec_test=tfidf_vectorizer.transform(x_test_first.values.astype('U'))
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(first_vec_train,y_train_first)

PassiveAggressiveClassifier(max_iter=50)

### Results

In [7]:
y_pred=pac.predict(first_vec_test)
score=accuracy_score(y_test_first,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.07%


In [8]:
confusion_matrix(y_test_first,y_pred)

array([[334,  26],
       [ 26, 364]])

In [9]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 92.63%


### Trying another dataset

In [10]:
df_true=pd.read_csv('True.csv', nrows=2500)
df_true['label']= 0
df_true_rep=[df_true['text'][i].replace('WASHINGTON (Reuters) - ','').replace('LONDON (Reuters) - ','').replace('(Reuters) - ','') for i in range(len(df_true['text']))]
df_true['text']=df_true_rep
df_fake = pd.read_csv("Fake.csv" ,nrows=2500)
df_fake['label']= 1
df_final=pd.concat([df_true,df_fake])
df_final=df_final.drop(['subject','date'], axis=1)

In [11]:
df_final.head(10)

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,0
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,0
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/President Donald Trump called on the U...,0
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./The White House said on ...",0
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla President Donald Trump sa...",0
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,0
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,0
9,Alabama official to certify Senator-elect Jone...,Alabama Secretary of State John Merrill said h...,0


In [12]:
df_final.tail(10)

,title,text,label
2490,REPORT: Even Republicans Are ‘Disturbed’ By T...,Donald Trump s days in the White House could b...,1
2491,Eric Trump Just Asked The Labor Department To...,"If any other American company did this, Donald...",1
2492,REPORT: Trump Considering Using National Guar...,Donald Trump reportedly wants to use the Natio...,1
2493,WATCH: Trump Assumes Black Reporter Can Set U...,After claiming that he s the least racist per...,1
2494,Trump Thinks Rush Limbaugh Is Real News And T...,Rush Limbaugh praised Donald Trump s insanely ...,1
2495,"WATCH: Fox News Host DEFENDS CNN Reporter, Te...",Even Fox News is calling Donald Trump out for ...,1
2496,GOP Senator DESPERATELY Worried About Trump’s...,"On Thursday afternoon, the world was treated t...",1
2497,WATCH: NBC Reporter Calls Trump Out For Lying...,Donald Trump finally held his first solo press...,1
2498,Tapper SLAMS Trump For BIZARRE Behavior At ‘W...,There is certainly no love lost between CNN an...,1
2499,LOL: Putin Is Angry Now Because Trump Gets Mo...,Vladimir Putin is super pissed because Russian...,1


### Model build and words vecrorize

In [13]:
x_train_second,x_test_second,y_train_second,y_test_second=train_test_split(df_final['text'], df_final['label'], test_size=0.25)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75 )

In [14]:
second_vec_train=tfidf_vectorizer.fit_transform(x_train_second.values.astype('U')) 
second_vec_test=tfidf_vectorizer.transform(x_test_second.values.astype('U'))
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(second_vec_train, y_train_second)

PassiveAggressiveClassifier(max_iter=50)

### Results

In [15]:
y_pred=pac.predict(second_vec_test)
score=accuracy_score(y_test_second,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.2%


In [16]:
confusion_matrix(y_test_second,y_pred)

array([[622,   5],
       [  5, 618]])

In [17]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 91.8%


### Try other method(Random Forest Classfier)

In [18]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(first_vec_train, y_train_first)

RandomForestClassifier(random_state=0)

In [19]:
y_pred=RFC.predict(first_vec_test)
score=accuracy_score(y_test_first,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 89.87%


In [20]:
confusion_matrix(y_test_first,y_pred)

array([[335,  25],
       [ 51, 339]])

In [21]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(RFC, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 88.53%


### Second dataset

In [22]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(second_vec_train, y_train_second)

RandomForestClassifier(random_state=0)

In [23]:
y_pred=RFC.predict(second_vec_test)
score=accuracy_score(y_test_second,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.2%


In [24]:
confusion_matrix(y_test_second,y_pred)

array([[621,   6],
       [  4, 619]])

In [25]:
X=tfidf_vectorizer.transform(df['text'].values.astype('U'))
scores = cross_val_score(RFC, X, df['label'].values, cv=5)
print(f'Accuracy: {round(scores.mean()*100,2)}%')

Accuracy: 88.53%


# Neural Networks

## Data preparation

In [26]:
df_train = df.sample(frac=.8, random_state=11)
df_test = df.drop(df_train.index)

In [27]:
y_train = df_train['label'].values
y_test = df_test['label'].values

In [28]:
X_train = df_train['text'].values
X_test = df_test['text'].values

In [29]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word
print(len(word_dict))

58498


In [30]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [114]:
X_train_pad = pad_sequences(X_train_seq, maxlen=512, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=512, padding='post')

In [119]:
laenge_pads = 512
vocab_size = len(word_dict)
output_dim=64

## Simple LSTM

In [129]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=anz_woerter+1, output_dim=output_dim, input_length=laenge_pads))
lstm_model.add(Dropout(0.1))
lstm_model.add(LSTM(128))
lstm_model.add(Dropout(0.1))
lstm_model.add(Dropout(0.1))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, 512, 64)           3743936   
_________________________________________________________________
dropout_70 (Dropout)         (None, 512, 64)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dropout_71 (Dropout)         (None, 128)               0         
_________________________________________________________________
dropout_72 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 129       
Total params: 3,842,881
Trainable params: 3,842,881
Non-trainable params: 0
___________________________________________

In [130]:
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/10
38/38 [==============================] - 22s 581ms/step - loss: 0.6550 - accuracy: 0.6254 - val_loss: 0.6272 - val_accuracy: 0.6400
Epoch 2/10
38/38 [==============================] - 22s 585ms/step - loss: 0.5937 - accuracy: 0.7325 - val_loss: 0.5948 - val_accuracy: 0.7233
Epoch 3/10
38/38 [==============================] - 22s 589ms/step - loss: 0.4819 - accuracy: 0.7912 - val_loss: 0.5179 - val_accuracy: 0.7433
Epoch 4/10
38/38 [==============================] - 22s 590ms/step - loss: 0.3689 - accuracy: 0.8413 - val_loss: 0.5250 - val_accuracy: 0.7333
Epoch 5/10
38/38 [==============================] - 23s 597ms/step - loss: 0.3111 - accuracy: 0.8562 - val_loss: 0.4981 - val_accuracy: 0.7800
Epoch 6/10
38/38 [==============================] - 23s 599ms/step - loss: 0.2991 - accuracy: 0.8721 - val_loss: 0.5345 - val_accuracy: 0.7367
Epoch 7/10
38/38 [==============================] - 23s 599ms/step - loss: 0.2790 - accuracy: 0.8621 - val_loss: 0.5506 - val_accuracy: 0.7800

In [131]:
pred_y = lstm_model.predict_classes(X_test_pad)
score=accuracy_score(y_test,pred_y)
print(f'PAC Accuracy: {round(score*100,2)}%')

PAC Accuracy: 81.33%


In [132]:
confusion_matrix(y_test,pred_y)

array([[279,  20],
       [ 92, 209]])

## Simple RNN

In [133]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim = vocab_size + 1, output_dim = output_dim, input_length = laenge_pads))
rnn_model.add(SimpleRNN(output_dim, return_sequences=True, kernel_constraint=maxnorm(50)))
rnn_model.add(Dropout(0.1))
rnn_model.add(SimpleRNN(output_dim, return_sequences=True, kernel_constraint=maxnorm(50)))
rnn_model.add(Dropout(0.1))
rnn_model.add(SimpleRNN(output_dim))
rnn_model.add(Dropout(0.1))
rnn_model.add(Dense(1,activation='sigmoid'))
rnn_model.compile(optimizer='sgd', loss='mse',  metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (None, 512, 64)           3743936   
_________________________________________________________________
simple_rnn_28 (SimpleRNN)    (None, 512, 64)           8256      
_________________________________________________________________
dropout_73 (Dropout)         (None, 512, 64)           0         
_________________________________________________________________
simple_rnn_29 (SimpleRNN)    (None, 512, 64)           8256      
_________________________________________________________________
dropout_74 (Dropout)         (None, 512, 64)           0         
_________________________________________________________________
simple_rnn_30 (SimpleRNN)    (None, 64)                8256      
_________________________________________________________________
dropout_75 (Dropout)         (None, 64)              

In [134]:
history = rnn_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/10
38/38 [==============================] - 15s 401ms/step - loss: 0.2473 - accuracy: 0.5829 - val_loss: 0.2406 - val_accuracy: 0.6233
Epoch 2/10
38/38 [==============================] - 13s 354ms/step - loss: 0.2387 - accuracy: 0.6171 - val_loss: 0.2362 - val_accuracy: 0.6317
Epoch 3/10
38/38 [==============================] - 13s 352ms/step - loss: 0.2384 - accuracy: 0.6129 - val_loss: 0.2345 - val_accuracy: 0.6267
Epoch 4/10
38/38 [==============================] - 14s 357ms/step - loss: 0.2337 - accuracy: 0.6246 - val_loss: 0.2353 - val_accuracy: 0.6300
Epoch 5/10
38/38 [==============================] - 14s 361ms/step - loss: 0.2444 - accuracy: 0.5758 - val_loss: 0.2392 - val_accuracy: 0.6217
Epoch 6/10
38/38 [==============================] - 12s 321ms/step - loss: 0.2430 - accuracy: 0.5813 - val_loss: 0.2638 - val_accuracy: 0.5183
Epoch 7/10
38/38 [==============================] - 12s 313ms/step - loss: 0.2536 - accuracy: 0.5483 - val_loss: 0.2426 - val_accuracy: 0.5767

In [135]:
pred_y = rnn_model.predict_classes(X_test_pad)
score=accuracy_score(y_test,pred_y)
print(f'PAC Accuracy: {round(score*100,2)}%')

PAC Accuracy: 62.33%


In [136]:
confusion_matrix(y_test,pred_y)

array([[203,  96],
       [130, 171]])

## Recurrent Convolutional Neural Networks

In [115]:
conv_lstm_model = Sequential()
conv_lstm_model.add(Embedding(input_dim=anz_woerter+1, output_dim=output_dim, input_length=laenge_pads))
conv_lstm_model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
conv_lstm_model.add(MaxPooling1D(pool_size=2))
conv_lstm_model.add(Conv1D(filters=512, kernel_size=5, padding='same', activation='relu'))
conv_lstm_model.add(MaxPooling1D(pool_size=2))
conv_lstm_model.add(Conv1D(filters=768, kernel_size=5, padding='same', activation='relu'))
conv_lstm_model.add(MaxPooling1D(pool_size=2))
conv_lstm_model.add(LSTM(128, return_sequences=True))
conv_lstm_model.add(Dropout(0.1))
conv_lstm_model.add(LSTM(128))
conv_lstm_model.add(Dropout(0.1))
conv_lstm_model.add(Dense(1, activation='sigmoid'))

conv_lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
conv_lstm_model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 512, 64)           3743936   
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 512, 256)          82176     
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 256, 256)          0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 256, 512)          655872    
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 128, 512)          0         
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 128, 768)          1966848   
_________________________________________________________________
max_pooling1d_33 (MaxPooling (None, 64, 768)         

In [116]:
history = conv_lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/10
38/38 [==============================] - 85s 2s/step - loss: 0.6607 - accuracy: 0.6108 - val_loss: 0.5827 - val_accuracy: 0.6683
Epoch 2/10
38/38 [==============================] - 86s 2s/step - loss: 0.4186 - accuracy: 0.8196 - val_loss: 0.5474 - val_accuracy: 0.7167
Epoch 3/10
38/38 [==============================] - 86s 2s/step - loss: 0.2141 - accuracy: 0.9075 - val_loss: 0.4816 - val_accuracy: 0.8167
Epoch 4/10
38/38 [==============================] - 86s 2s/step - loss: 0.1198 - accuracy: 0.9683 - val_loss: 1.9004 - val_accuracy: 0.5633
Epoch 5/10
38/38 [==============================] - 87s 2s/step - loss: 0.3142 - accuracy: 0.8933 - val_loss: 0.4236 - val_accuracy: 0.8550
Epoch 6/10
38/38 [==============================] - 87s 2s/step - loss: 0.0663 - accuracy: 0.9833 - val_loss: 0.5138 - val_accuracy: 0.8617
Epoch 7/10
38/38 [==============================] - 90s 2s/step - loss: 0.0474 - accuracy: 0.9875 - val_loss: 0.4451 - val_accuracy: 0.8867
Epoch 8/10
38/38 [==

In [117]:
pred_y = conv_lstm_model.predict_classes(X_test_pad)
score=accuracy_score(y_test,pred_y)
print(f'PAC Accuracy: {round(score*100,2)}%')

PAC Accuracy: 88.0%


In [118]:
confusion_matrix(y_test,pred_y)

array([[276,  23],
       [ 49, 252]])